In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp pipeline

In [ ]:
# hide

from kaggle_m5_nbdev.core import test_eq, test_err, configure_logging
log = configure_logging('./tmp', 'test_log', con_log_lvl='DEBUG')

In [ ]:
#export

from kaggle_m5_nbdev.core import read_series_sample, melt_sales_series, extract_day_ids, join_w_calendar, join_w_prices
from kaggle_m5_nbdev.core import to_parquet, get_submission_template_melt
from kaggle_m5_nbdev.petastorm import ParquetIterableDataset
import os

def prepare_data_on_disk(log, n_sample_series, processed_dir, raw_dir, force_data_prep):
    expected_path = f'{processed_dir}/sales_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    log.info(f'Not found parquet file ({expected_path}) - preparing the data')

    sales_series = read_series_sample(log, n_sample_series)
    sales_series = melt_sales_series(sales_series)
    sales_series = extract_day_ids(sales_series)
    sales_series = join_w_calendar(sales_series, raw_dir)
    sales_series = join_w_prices(sales_series, raw_dir).persist()
    to_parquet(sales_series, 'sales_series_melt.parquet', processed_dir, log)

In [ ]:
prepare_data_on_disk(log, n_sample_series=10, processed_dir='./tmp', raw_dir='raw', force_data_prep=True)
prepare_data_on_disk(log, n_sample_series=10, processed_dir='./tmp', raw_dir='raw', force_data_prep=False)

 INFO [2020-07-30 05:57:00] root:  Not found parquet file (./tmp/sales_series_melt.parquet) - preparing the data
DEBUG [2020-07-30 05:57:06] root:   Read 10 series
DEBUG [2020-07-30 05:57:18] root:   Setting index
DEBUG [2020-07-30 05:57:18] root:   Setting index - done
DEBUG [2020-07-30 05:57:18] root:   Dropping 'day_date' from Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'day_id',
       'sales', 'day_date', 'day_date_str', 'month_id', 'date', 'wm_yr_wk',
       'snap_flag', 'sell_price', 'sales_dollars'],
      dtype='object')
DEBUG [2020-07-30 05:57:18] root:   Encoding: id
DEBUG [2020-07-30 05:57:18] root:   Encoding: item_id
DEBUG [2020-07-30 05:57:18] root:   Encoding: dept_id
DEBUG [2020-07-30 05:57:18] root:   Encoding: cat_id
DEBUG [2020-07-30 05:57:19] root:   Encoding: store_id
DEBUG [2020-07-30 05:57:19] root:   Encoding: state_id
DEBUG [2020-07-30 05:57:20] root:   Encoding: day_date_str
DEBUG [2020-07-30 05:57:20] root:   Encoding: date
DEBUG [20

In [ ]:
# TODO: move to core (don't forget to add import here)

from sklearn.preprocessing import LabelEncoder
import dask.dataframe as dd
import numpy as np

def load_encoders(processed):
    def _load(fn):
        l = LabelEncoder()
        l.classes_ = np.load(f'{processed}/{fn}', allow_pickle=True)
        return l

    encoders_paths = filter(lambda p: p.endswith('.npy'), os.listdir(processed))
    encoders = {fn[:-len('.npy')]:_load(fn) for fn in encoders_paths}

    return encoders

def encode(log, me, processed):
    encoders = load_encoders(processed)
    continuous_cols = ['sell_price']

    for col in me.columns:
        dtype_str = str(me[col].dtype)
        if col in continuous_cols:
            log.debug(f"Encoding {col} ({dtype_str}) as float32 just in case for pytorch")
            me[col] = me[col].astype('float32')
            continue

        log.debug(f"Encoding {col} ({dtype_str}) as categorical ")

        unlabelable = ~me[col].isin(encoders[col].classes_)
        unlabelable_count = unlabelable.sum()
        if unlabelable_count > 0:
            default_label = encoders[col].classes_[0]
            log.warning(f"{unlabelable_count} entries for {col} can't be labeled. Defaulting to {default_label} e.g.\n {me[unlabelable][col][:3].values}")
            me.loc[unlabelable, col] = default_label

        me[col] = encoders[col].transform(me[col])

    return me

In [ ]:
raw = 'raw'
processed = './tmp'

In [ ]:
def prepare_test_data_on_disk(log, raw, processed, force_data_prep):
    expected_path = f'{processed}/test_series_melt.parquet'
    if os.path.exists(expected_path) and not force_data_prep:
        log.info(f'Found parquet file ({expected_path})- skipping the prep')
        return

    template = get_submission_template_melt(raw)
    test_data = encode(log, template, processed)
    to_parquet(test_data, 'test_series_melt.parquet', processed, log)

In [ ]:
prepare_test_data_on_disk(log, raw='raw', processed='./tmp', force_data_prep=True)
prepare_test_data_on_disk(log, raw='raw', processed='./tmp', force_data_prep=False)

DEBUG [2020-07-30 05:57:40] root:   Encoding id (object) as categorical 
WARNING [2020-07-30 05:57:41] root:   1707160 entries for id can't be labeled. Defaulting to FOODS_3_551_CA_3_validation e.g.
 ['HOBBIES_1_001_CA_1_validation' 'HOBBIES_1_002_CA_1_validation'
 'HOBBIES_1_003_CA_1_validation']
DEBUG [2020-07-30 05:57:41] root:   Encoding date (object) as categorical 
WARNING [2020-07-30 05:57:42] root:   1707440 entries for date can't be labeled. Defaulting to 2011-01-29 e.g.
 [datetime.date(2016, 6, 20) datetime.date(2016, 6, 20)
 datetime.date(2016, 6, 20)]
DEBUG [2020-07-30 05:57:42] root:   Encoding sell_price (float64) as float32 just in case for pytorch
DEBUG [2020-07-30 05:57:42] root:   Encoding cat_id (object) as categorical 
DEBUG [2020-07-30 05:57:43] root:   Encoding dept_id (object) as categorical 
WARNING [2020-07-30 05:57:43] root:   343840 entries for dept_id can't be labeled. Defaulting to FOODS_3 e.g.
 ['FOODS_1' 'FOODS_1' 'FOODS_1']
DEBUG [2020-07-30 05:57:43] ro

In [ ]:
from torch.utils.data import TensorDataset, DataLoader as TorchDataLoader, IterableDataset
from collections import OrderedDict
def setup_data_loaders(processed, log):
    batch = 128

    train_ds = ParquetIterableDataset(f'file:{processed}/sales_series_melt.parquet', log, '.*part.(?!1).*')
    valid_ds = ParquetIterableDataset(f'file:{processed}/sales_series_melt.parquet', log, '.*part.1.*')
    test_ds  = ParquetIterableDataset(f'file:{processed}/test_series_melt.parquet', log)

    train_dl = TorchDataLoader(train_ds, batch_size=batch, shuffle=False, num_workers=0, drop_last=False)
    valid_dl = TorchDataLoader(valid_ds, batch_size=batch, shuffle=False, num_workers=0, drop_last=False)
    test_dl  = TorchDataLoader(test_ds,  batch_size=batch, shuffle=False, num_workers=0, drop_last=False)

    data = OrderedDict()
    data["train"] = train_dl
    data["valid"] = valid_dl
    data["test"]  = test_dl

    return data

In [ ]:
dls = setup_data_loaders(processed=processed, log=log)

DEBUG [2020-07-30 06:01:19] root: file:./tmp/sales_series_melt.parquet -> 1 files out of 4 remained after applying filter (.*part.(?!1).*)
DEBUG [2020-07-30 06:01:19] root: file:./tmp/sales_series_melt.parquet -> 1 files out of 4 remained after applying filter (.*part.1.*)


In [ ]:
def log_batch(model, data, log, log_stage):
   for key, dl in data.items():
        batch = next(iter(dl))
        log.debug(f"{key} model out @ {log_stage} {model.forward(batch['features']).transpose(1, 0)}")

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

from catalyst.dl import SupervisedRunner
from catalyst.utils import set_global_seed

class Net(nn.Sequential):
    def __init__(self, num_features):
        layers = []
        layer_dims = [num_features, 200,200,20,20,1]
        for in_features, out_features in zip(layer_dims[:-1], layer_dims[1:]):
            l = nn.Linear(in_features, out_features)
            # Note to self: loss @ init is quite important!
            torch.nn.init.xavier_uniform_(l.weight) 
            torch.nn.init.zeros_(l.bias)

            layers.append(l)
            layers.append(nn.ReLU())
        super(Net, self).__init__(*layers)

class MyLoss(nn.MSELoss):
    def __init__(self):
        super(MyLoss, self).__init__()

    def forward(self, inp, target):
        return super().forward(inp, target)

def do_train(data, log, log_dir):
    model = Net(num_features = 2)
    runner = SupervisedRunner()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    criterion = MyLoss()

    log_batch(model, data, log, "init")

    log.debug("Starting training")
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=data,
        logdir=f"{log_dir}/run",
        load_best_on_end=True,
        num_epochs=1)

    log_batch(model, data, log, "exit")

In [ ]:
do_train(dls, log, log_dir='./tmp')

DEBUG [2020-07-30 06:04:50] root:       Iterator created on file:./tmp/sales_series_melt.parquet
DEBUG [2020-07-30 06:04:52] root:   train model out @ init tensor([[0.1658, 0.1949, 0.1874, 0.1874, 0.0416, 0.1658, 0.1949, 0.0825, 0.0825,
         0.1824, 0.0825, 0.1658, 0.1824, 0.1949, 0.1974, 0.1949, 0.1824, 0.0825,
         0.1824, 0.1658, 0.1949, 0.1949, 0.1824, 0.0825, 0.0825, 0.1658, 0.1824,
         0.1658, 0.0825, 0.1949, 0.1658, 0.1949, 0.1949, 0.1949, 0.1824, 0.0825,
         0.1658, 0.0825, 0.1874, 0.1949, 0.1658, 0.1824, 0.1658, 0.0825, 0.0825,
         0.1658, 0.1658, 0.0825, 0.1658, 0.1658, 0.0825, 0.0825, 0.1949, 0.0825,
         0.0825, 0.1658, 0.0825, 0.0825, 0.0825, 0.1824, 0.1658, 0.1949, 0.1658,
         0.0825, 0.1949, 0.1666, 0.1658, 0.1658, 0.1949, 0.1949, 0.0825, 0.1949,
         0.0825, 0.0825, 0.0825, 0.1949, 0.1824, 0.1658, 0.0825, 0.1874, 0.0825,
         0.1658, 0.1824, 0.1949, 0.1658, 0.1658, 0.0825, 0.1949, 0.0825, 0.0825,
         0.1824, 0.1658, 0.1949, 0

[2020-07-30 06:06:29,955] 
1/1 * Epoch 1 (_base): lr=0.0100 | momentum=0.9000
1/1 * Epoch 1 (train): loss=56.5099
1/1 * Epoch 1 (valid): loss=87.0118
1/1 * Epoch 1 (test): loss=143.3571


DEBUG [2020-07-30 06:06:30] root:       Iterator created on file:./tmp/sales_series_melt.parquet


Top best models:
tmp/run/checkpoints/train.1.pth	87.0118
=> Loading checkpoint tmp/run/checkpoints/best_full.pth
loaded state checkpoint tmp/run/checkpoints/best_full.pth (global epoch 1, epoch 1, stage train)


DEBUG [2020-07-30 06:06:31] root:   train model out @ exit tensor([[8.7125, 3.5739, 9.0780, 3.5739, 7.6970, 3.5739, 9.0780, 3.5739, 3.5739,
         3.5739, 9.0780, 7.6564, 3.5739, 3.5739, 7.6970, 9.0780, 9.0780, 3.5739,
         3.5739, 7.6564, 9.0780, 9.0780, 7.6564, 7.6564, 3.5739, 3.5739, 7.6564,
         7.6564, 9.0780, 3.5739, 3.5739, 9.0780, 8.4688, 9.0780, 7.6564, 9.0780,
         9.0780, 3.5739, 9.0780, 9.0780, 9.0780, 3.5739, 8.4688, 3.5739, 8.4688,
         8.7125, 9.0780, 3.5739, 3.5739, 9.0780, 3.5739, 3.5739, 8.4688, 7.6564,
         9.0780, 9.0780, 9.0780, 9.0780, 7.6564, 8.4688, 7.6564, 8.4688, 3.5739,
         3.5739, 9.0780, 3.5739, 7.6564, 9.0780, 9.0780, 3.5739, 9.0780, 3.5739,
         7.6564, 3.5739, 7.6564, 3.5739, 9.0780, 3.5739, 7.6564, 3.5739, 3.5739,
         9.0780, 7.6564, 3.5739, 3.5739, 9.0780, 3.5739, 3.5739, 3.5739, 3.5739,
         9.0780, 3.5739, 8.4688, 8.4688, 3.5739, 7.6564, 7.6564, 7.6564, 7.6564,
         8.4688, 3.5739, 9.0780, 7.6564, 7.6564, 9